In [61]:
import numpy as np

In [297]:
def pd_projection(A):
    U,S,Vt = np.linalg.svd(A+A.T, full_matrices=True)
    S = np.maximum(S, 1e-3)
    return np.dot(U, np.dot(np.diag(S), Vt))

In [298]:
def kron_to_vec(S, m, n):
    Sigma = np.zeros((m*n,m*n))
    for i in range(m):
        for j in range(n):
            Sigma[:,i+j*m] = S_A[i*m:(i+1)*m,j*n:(j+1)*n].flatten('F')
    return Sigma

def vec_to_kron(Sigma, m, n):
    S = np.zeros((m**2,n**2))
    for i in range(m):
        for j in range(n):
            S[i*m:(i+1)*m,j*n:(j+1)*n] = Sigma[:,i+j*m].reshape(m,n,order='F')
    return S

In [370]:
class MatrixGaussian():
    def __init__(self, M, S=None, Sigma=None):
        self.M = M
        self.m, self.n = M.shape
        if S is None and Sigma is None:
            raise ArgumentError('Must provide covariance.')
        if S is None:
            self.Sigma = Sigma
            self.S = vec_to_kron(Sigma, self.m, self.n)
        elif Sigma is None:
            self.S = S
            self.Sigma = kron_to_vec(S, self.m, self.n)
        
    def sample(self):
        vect = np.random.multivariate_normal(self.M.flatten('F'),self.Sigma)
        return vect.reshape(self.m,self.n,order='F')

In [446]:
n , m= 2,3
mg = MatrixGaussian(np.zeros((2,2)),Sigma=np.eye(4))

In [447]:
print (mg.Sigma)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


In [448]:
print (mg.S)

[[1. 0. 0. 1.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [1. 0. 0. 1.]]


In [449]:
n, m = 2,3
L = 1e-1*np.random.randn(n*(n+m),n*(n+m))
mg = MatrixGaussian(np.zeros((n,n+m)),Sigma=np.dot(L, L.T) + 1e-3*np.eye(n*(n+m)))
S_BA = np.zeros((n**2, n*m))
for i in range(m):
    S_BA[:,i*n:(i+1)*n] = mg.S[:,(n+m)*n+i*(n+m):(n+m)*n+i*(n+m)+n]

In [450]:
np.set_printoptions(precision=6)
mg.S[:,(n+m)*n:]

array([[-3.120180e-02, -1.598883e-02,  1.399280e-01,  1.319147e-02,
        -5.265172e-02, -1.307329e-03, -1.585307e-02,  1.319147e-02,
         3.531956e-02,  9.910368e-03,  1.830406e-02, -3.739716e-02,
        -5.265172e-02,  9.910368e-03,  1.064510e-01],
       [ 5.195674e-02,  1.023038e-02,  1.427300e-02, -8.470199e-03,
         2.379581e-02,  3.752602e-02, -1.531034e-02, -2.531444e-03,
        -4.738518e-03, -2.801969e-03, -1.533811e-02,  1.514831e-03,
        -5.033511e-02, -1.666393e-02,  2.401458e-02],
       [-7.035780e-03, -2.564285e-02,  1.427300e-02, -2.531444e-03,
        -5.033511e-02,  1.395571e-02,  1.697478e-03, -8.470199e-03,
        -4.738518e-03, -1.666393e-02,  6.583021e-02, -7.448532e-02,
         2.379581e-02, -2.801969e-03,  2.401458e-02],
       [ 1.008758e-04,  3.976280e-03,  7.068932e-02,  3.488166e-02,
         2.255779e-02, -2.946147e-02, -2.869972e-04,  3.488166e-02,
         5.586253e-02,  1.509520e-02, -6.526142e-03,  1.768028e-02,
         2.255779e-02,

In [451]:
S_BA

array([[-0.031202, -0.015989, -0.001307, -0.015853,  0.018304, -0.037397],
       [ 0.051957,  0.01023 ,  0.037526, -0.01531 , -0.015338,  0.001515],
       [-0.007036, -0.025643,  0.013956,  0.001697,  0.06583 , -0.074485],
       [ 0.000101,  0.003976, -0.029461, -0.000287, -0.006526,  0.01768 ]])

In [452]:
C = np.eye(n)

In [453]:
avg = 0.
for _ in range(1000):
    samp = mg.sample()
    A = samp[:,:n]
    B = samp[:,n:]
    avg += np.dot(A.T, np.dot(C, B))/1000.

In [454]:
avg

array([[-0.034037, -0.026579,  0.008243],
       [-0.011081, -0.013376, -0.018687]])

In [455]:
np.dot(S_BA.T, C.flatten('F')).reshape(n,m,order='F')

array([[-0.031101, -0.030769,  0.011778],
       [-0.012013, -0.01614 , -0.019717]])

In [440]:
A = np.random.randn(n,n)
s = np.eye(n)+np.random.randn(n,n)*1e-1
avg = 0.
for _ in range(100):
    x = np.random.multivariate_normal(np.zeros(n),s)
    avg += np.dot(x, np.dot(A, x))/100

/Users/shane/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: covariance is not positive-semidefinite.
  """


In [441]:
avg

-0.8044908206944809

In [442]:
np.trace(np.dot(A,s))

-0.43100535927026695